###   Introducción a [Apache Spark](https://spark.apache.org/) con el API de pySpark  [pySpark API](https://spark.apache.org/docs/latest/api/python/pyspark.html#pyspark.RDD)

#### **Spark Context**
#### Para usar Spark y su API, necesitaremos usar `SparkContext`. Al ejecutar Spark, es necesario crear un nuevo objeto del tipo [SparkContext](http://spark.apache.org/docs/latest/api/python/pyspark.html#pyspark.SparkContext).  Cuando se crea el `SparkContext`, pide al nodo maestro una seríe de núcleos para hacer el trabajo. Cuando utilizamos la consola interactiva, el `SparkContext` se crea automáticamente como ` sc`.

#### Acceder a los programas de controlador Spark a través de un objeto SparkContext, que representa una conexión a un clúster de computación. Un objeto de contexto Spark (`sc`) es el punto de entrada principal para la funcionalidad Spark. Un contexto Spark se puede utilizar para crear conjuntos de datos distribuidos resilientes (RDD) en un clúster.

In [1]:
# Spark Context sc
type(sc)

pyspark.context.SparkContext

#### `SparkContext` atributos
####  [dir()](https://docs.python.org/2/library/functions.html?highlight=dir#dir) funcion permite obtener una lista de los atributos accesibles a través del objeto sc

In [2]:
# Lista de atributos
dir(sc)

['PACKAGE_EXTENSIONS',
 '__class__',
 '__delattr__',
 '__dict__',
 '__doc__',
 '__enter__',
 '__exit__',
 '__format__',
 '__getattribute__',
 '__getnewargs__',
 '__hash__',
 '__init__',
 '__module__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_accumulatorServer',
 '_active_spark_context',
 '_batchSize',
 '_callsite',
 '_checkpointFile',
 '_conf',
 '_dictToJavaMap',
 '_do_init',
 '_ensure_initialized',
 '_gateway',
 '_getJavaStorageLevel',
 '_initialize_context',
 '_javaAccumulator',
 '_jsc',
 '_jvm',
 '_lock',
 '_next_accum_id',
 '_pickled_broadcast_vars',
 '_python_includes',
 '_temp_dir',
 '_unbatched_serializer',
 'accumulator',
 'addFile',
 'addPyFile',
 'appName',
 'applicationId',
 'binaryFiles',
 'binaryRecords',
 'broadcast',
 'cancelAllJobs',
 'cancelJobGroup',
 'defaultMinPartitions',
 'defaultParallelism',
 'dump_profiles',
 'emptyRDD',
 'environment',
 'getConf',
 'getLocalProp

In [3]:
# Ayuda sobre el objeto sc
help(sc)

Help on SparkContext in module pyspark.context object:

class SparkContext(__builtin__.object)
 |  Main entry point for Spark functionality. A SparkContext represents the
 |  connection to a Spark cluster, and can be used to create L{RDD} and
 |  broadcast variables on that cluster.
 |  
 |  Methods defined here:
 |  
 |  __enter__(self)
 |      Enable 'with SparkContext(...) as sc: app(sc)' syntax.
 |  
 |  __exit__(self, type, value, trace)
 |      Enable 'with SparkContext(...) as sc: app' syntax.
 |      
 |      Specifically stop the context on exit of the with block.
 |  
 |  __getnewargs__(self)
 |  
 |  __init__(self, master=None, appName=None, sparkHome=None, pyFiles=None, environment=None, batchSize=0, serializer=PickleSerializer(), conf=None, gateway=None, jsc=None, profiler_cls=<class 'pyspark.profiler.BasicProfiler'>)
 |      Create a new SparkContext. At least the master and app name should be set,
 |      either through the named parameters here or through C{conf}.
 |   

In [4]:
# ver la versión de spark
sc.version

u'2.1.0'

#### **Creando nuestro primer RDD**
#### En Spark, primero tenemos que definir un  [Resilient Distributed Dataset](http://spark.apache.org/docs/latest/api/python/pyspark.html#pyspark.RDD) (RDD). Entonces podemos aplicar una o más transformaciones a un RDD. * Un RDD es inmutable, por lo que una vez que se crea, no se puede cambiar * Como resultado, cada transformación crea un nuevo RDD. Finalmente, podemos aplicar una o más acciones a los RDDs. Tenga en cuenta que Spark utiliza evaluación perezosa, por lo que las transformaciones no se ejecutan hasta que se produce una acción.
#### Realizaremos varios ejercicios para obtener una mejor comprensión de los RDD:
* ##### Crear una colección de 100 enteros
* ##### Crear un RDD a partir de esta colección
* ##### Sumar uno a cada elemento mediante la funcion map
* ##### Operación collect para ver los resultados
* ##### Operación `count' para ver el número de elementos
* ##### Aplicar la transformación `filter` y ver los resultados con ` collect`
* ##### Filtro mediante funciones Lambda
* ##### Operaciones RDD
* ##### Cache y opciones de almacenamient

#### **Crear una colección de 100 enteros**
#### Usaremos la función [xrange()](https://docs.python.org/2/library/functions.html?highlight=xrange#xrange) para crear una lista [list()](https://docs.python.org/2/library/functions.html?highlight=list#list) de enteros.

In [5]:
from numpy import *
data = xrange(0, 100)

In [6]:
len(data)

100

### **Distribución de datos y uso de una colección para crear un RDD**
#### En Spark, los conjuntos de datos se representan como una lista de entradas, donde la lista se divide en muchas particiones diferentes que se almacenan cada una en una máquina diferente. Cada partición contiene un subconjunto único de las entradas de la lista. En Spark estos conjuntos de datos se llaman "Resilient Distributed Datasets" (RDDs).
#### Una de las características definitorias de Spark, en comparación con otras estructuras de análisis de datos (por ejemplo, Hadoop), es que almacena los datos en la memoria en lugar de en el disco. Esto permite que las aplicaciones Spark se ejecuten mucho más rápidamente, ya que no se ralentizan al necesitar leer los datos del disco.
#### La siguiente figura ilustra cómo Spark distribuye una lista de entradas de datos en particiones que están almacenadas en memoria
![partitions](http://spark-mooc.github.io/web-assets/images/partitions.png)
#### Para crear el RDD, usamos el método `sc.parallelize()`, que le dice a Spark que cree un nuevo conjunto de datos de entrada basados en los datos que se transmiten. En este ejemplo, proporcionaremos un `xrange`.El segundo argumento de [sc.parallelize()](http://spark.apache.org/docs/latest/api/python/pyspark.html#pyspark.SparkContext.parallelize) establece cuántas particiones se utilizan para dividir los datos en cuanto almacena los datos en la memoria. Tenga en cuenta que para un mejor rendimiento al usar `parallelize`, se recomienda `xrange () `si la entrada representa un rango.
#### Hay varios tipos de RDDs.  La clase base de los RDD es [pyspark.RDD](http://spark.apache.org/docs/latest/api/python/pyspark.html#pyspark.RDD) . Ya que todos los tipos heredan de  `pyspark.RDD` comparten la misma API.  Podemos ver  que `sc.parallelize ()` genera un `pyspark.rdd.PipelinedRDD` cuando su entrada es un xrange, y un `pyspark.RDD`  cuando su entrada es un range

In [7]:
# Parallelize usando 8 particiones
xrangeRDD = sc.parallelize(data, 8)

In [8]:
# ayuda del método parallelize
help(sc.parallelize)

Help on method parallelize in module pyspark.context:

parallelize(self, c, numSlices=None) method of pyspark.context.SparkContext instance
    Distribute a local Python collection to form an RDD. Using xrange
    is recommended if the input represents a range for performance.
    
    >>> sc.parallelize([0, 2, 3, 4, 6], 5).glom().collect()
    [[0], [2], [3], [4], [6]]
    >>> sc.parallelize(xrange(0, 6, 2), 5).glom().collect()
    [[], [0], [], [2], [4]]



In [9]:
# Tipo de datos de sc.parallelize() --> xrange--> pyspark.rdd.PipelinedRDD
print('tipo dato xrangeRDD: {0}'.format(type(xrangeRDD)))

# si usamos un range,el tipo devuelto por parallelize es pyspark.RDD
dataRange = range(0, 100)
rangeRDD = sc.parallelize(dataRange, 8)
print('type dato dataRangeRDD: {0}'.format(type(rangeRDD)))

tipo dato xrangeRDD: <class 'pyspark.rdd.PipelinedRDD'>
type dato dataRangeRDD: <class 'pyspark.rdd.RDD'>


In [10]:
# comprobar los tipos de datos
# con help podemos ver los métodos que podemos llamar en este RDD
help(xrangeRDD)

Help on PipelinedRDD in module pyspark.rdd object:

class PipelinedRDD(RDD)
 |  Pipelined maps:
 |  
 |  >>> rdd = sc.parallelize([1, 2, 3, 4])
 |  >>> rdd.map(lambda x: 2 * x).cache().map(lambda x: 2 * x).collect()
 |  [4, 8, 12, 16]
 |  >>> rdd.map(lambda x: 2 * x).map(lambda x: 2 * x).collect()
 |  [4, 8, 12, 16]
 |  
 |  Pipelined reduces:
 |  >>> from operator import add
 |  >>> rdd.map(lambda x: 2 * x).reduce(add)
 |  20
 |  >>> rdd.flatMap(lambda x: [x, x]).reduce(add)
 |  20
 |  
 |  Method resolution order:
 |      PipelinedRDD
 |      RDD
 |      __builtin__.object
 |  
 |  Methods defined here:
 |  
 |  __init__(self, prev, func, preservesPartitioning=False)
 |  
 |  getNumPartitions(self)
 |  
 |  id(self)
 |  
 |  ----------------------------------------------------------------------
 |  Methods inherited from RDD:
 |  
 |  __add__(self, other)
 |      Return the union of this RDD and another one.
 |      
 |      >>> rdd = sc.parallelize([1, 1, 2, 3])
 |      >>> (rdd + r

In [11]:
#  con help podemos ver los métodos que podemos llamar en este RDD
help(rangeRDD)

Help on RDD in module pyspark.rdd object:

class RDD(__builtin__.object)
 |  A Resilient Distributed Dataset (RDD), the basic abstraction in Spark.
 |  Represents an immutable, partitioned collection of elements that can be
 |  operated on in parallel.
 |  
 |  Methods defined here:
 |  
 |  __add__(self, other)
 |      Return the union of this RDD and another one.
 |      
 |      >>> rdd = sc.parallelize([1, 1, 2, 3])
 |      >>> (rdd + rdd).collect()
 |      [1, 1, 2, 3, 1, 1, 2, 3]
 |  
 |  __getnewargs__(self)
 |  
 |  __init__(self, jrdd, ctx, jrdd_deserializer=AutoBatchedSerializer(PickleSerializer()))
 |  
 |  __repr__(self)
 |  
 |  aggregate(self, zeroValue, seqOp, combOp)
 |      Aggregate the elements of each partition, and then the results for all
 |      the partitions, using a given combine functions and a neutral "zero
 |      value."
 |      
 |      The functions C{op(t1, t2)} is allowed to modify C{t1} and return it
 |      as its result value to avoid object allocat

In [12]:
# Obtener el número de particiones llamando al método getNumPartitions()
xrangeRDD.getNumPartitions()

8

In [13]:
print(xrangeRDD.collect())

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99]


### **Añadir uno a cada valor mediante la funcion `map`**
#### Hasta ahora, hemos creado un dataset distribuido que se divide en muchas particiones, donde cada partición se almacena en una sola máquina en nuestro clúster. Veamos lo que sucede cuando realizamos una operación básica en el conjunto de datos. Muchas operaciones útiles de análisis de datos se pueden especificar como "hacer algo a cada elemento del conjunto de datos". Estas operaciones de datos paralelos son convenientes porque cada elemento del conjunto de datos puede procesarse individualmente: la operación en una entrada no afecta las operaciones en ninguna de las otras entradas. Por lo tanto, Spark puede paralelizar la operación.
#### `map(f)`, es la transformación de Spark más común: aplica una función `f` a cada elemento del conjunto de datos y genera el conjunto de datos resultante. Cuando se ejecuta `map()` en un conjunto de datos, se inicia una * etapa * de tareas. A * stage * es un grupo de tareas que realizan el mismo cálculo, pero con diferentes datos de entrada. Se inicia una tarea para cada partición, como se muestra en el ejemplo siguiente. Una tarea es una unidad de ejecución que se ejecuta en una sola máquina. Cuando ejecutamos `map(f)` dentro de una partición, una nueva * tarea * aplica `f` a todas las entradas de una partición en particular y genera una nueva partición. En esta figura de ejemplo, el conjunto de datos se divide en cuatro particiones, por lo que se ejecutan cuatro tareas `map ()`.
![tasks](http://spark-mooc.github.io/web-assets/images/tasks.png)
#### Al aplicar la transformación `map()`, cada elemento del RDD principal se asignará a un elemento en el nuevo RDD. Por lo tanto, si el RDD padre tiene veinte elementos, el nuevo RDD también tendrá veinte elementos.
#### Ahora usaremos `map()` para sumar uno a cada valor en el RDD. En primer lugar, definimos una función Python llamada `add()` que suma el elemento de entrada. En segundo lugar, pasaremos cada elemento en el RDD base a una transformación `map()` que aplique la función `add()` a cada elemento. Y finalmente, imprimimos la jerarquía de transformación RDD usando `toDebugString()`.

In [14]:
# Creamos una función que sume 1
def add(value):
    return (value + 1)

# Transformar xrangeRDD a través de la función map utilizando la función add
addRDD = xrangeRDD.map(add)

print(addRDD.toDebugString())

(8) PythonRDD[3] at RDD at PythonRDD.scala:48 []
 |  ParallelCollectionRDD[0] at parallelize at PythonRDD.scala:475 []


### ** Operacíon `collect` para ver los resultados **
#### `Collect()` se utiliza normalmente para ver los resultados después de aplicar un filtro u otra operación para asegurarse de que sólo estamos devolviendo una * pequeña * cantidad de datos. Esto se hace porque los datos devueltos deben ajustarse a la memoria disponible.
#### El método `collect()` es la primera operación de acción que podemos ejecutar. Las operaciones de acción hacen que Spark realice las operaciones de transformación necesarias para calcular el RDD devuelto por la acción. En nuestro ejemplo, esto significa que ahora se lanzarán tareas para realizar las operaciones `parallelize`,` map`, y `collect`.
#### En este ejemplo, el conjunto de datos se divide en cuatro particiones, por lo que se ejecutan cuatro tareas `collect()`. Cada tarea recoge las entradas en su partición y envía el resultado al SparkContext, que crea una lista de los valores.
#### Ahora vamos a ejecutar `collect()` en `subRDD`.

In [15]:
# Let's collect the data
print addRDD.collect()

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100]


### ** Realiza la acción `count` para ver el número de elementos **
#### Uno de los trabajos más básicos que podemos ejecutar es el método `count ()` que contará el número de elementos en un RDD. Dado que `map ()` crea un nuevo RDD con el mismo número de elementos que el RDD inicial, esperamos que aplicar `count ()` a cada RDD devolverá el mismo resultado.
#### Tenga en cuenta que dado que `count ()` es una operación de acción, si no hubiéramos realizado una acción con `collect ()`, entonces Spark realizaría las operaciones de transformación cuando ejecutamos `count ()`.

In [16]:
print xrangeRDD.count()
print addRDD.count()

100
100


### ** Aplicar la transformación `filter` y ver los resultados con ` collect` **
#### A continuación, crearemos un nuevo RDD que sólo contenga los valores inferiores a diez utilizando la operación `filter(f)`. El método `filter (f)` es una operación de transformación que crea un nuevo RDD a partir de la entrada RDD aplicando la función de filtro `f` a cada elemento del RDD principal y sólo pasa aquellos elementos en los que la función filter devuelve` True`. Los elementos que no devuelvan True serán eliminados. Al igual que `map()`, el filtro puede aplicarse individualmente a cada entrada del dataset, por lo que es fácilmente paralelizable usando Spark.

#### Para filtrar este conjunto de datos, definiremos una función llamada `filtro()`, que devuelve `True` si la entrada es menor de 10 y` False` en caso contrario. Esta función se pasará a la transformación `filter()` como la función de filtro `f`.
#### Para ver la lista filtrada de elementos menos de diez, necesitamos crear una nueva lista en el controlador de los datos distribuidos en los nodos ejecutor. Utilizamos el método `collect()` para devolver una lista que contiene todos los elementos de este RDD con los valores filtrados

In [17]:
def filtro(value):
    if (value < 10):
        return True
    else:
        return False

# Pasamos la función al método filter para transformar un RDD en otro
filteredRDD = addRDD.filter(filtro)

# resultados después de aplicar el filtro
print filteredRDD.collect()

[1, 2, 3, 4, 5, 6, 7, 8, 9]


### ** Filtro mediante funciones Lambda**

#### Python admite el uso de funciones anónimas de una línea que no están enlazadas a un nombre en tiempo de ejecución. Tomados de LISP, estas funciones `lambda` pueden utilizarse donde quiera que se requieran objetos de función. Están sintácticamente restringidos a una única expresión. En el caso anterior hemos definido una simple función , pero usar una función `lambda()` es una forma equivalente y más compacta de codificación.
#### Aquí, en lugar de definir una función separada para la transformación `filter()`, usaremos una función inline `lambda()`.

In [18]:
#obtenemos el mismo resulatdo que antes
lambdaRDD = addRDD.filter(lambda x: x < 10)
lambdaRDD.collect()

[1, 2, 3, 4, 5, 6, 7, 8, 9]

In [19]:
# Obtener valores pares del conjunto de datos anterior
paresRDD = lambdaRDD.filter(lambda x: x % 2 == 0)
paresRDD.collect()

[2, 4, 6, 8]

### ** Operaciones RDD **

#### Operaciones a revisar: [first()](http://spark.apache.org/docs/latest/api/python/pyspark.html#pyspark.RDD.first), [take()](http://spark.apache.org/docs/latest/api/python/pyspark.html#pyspark.RDD.take), [top()](http://spark.apache.org/docs/latest/api/python/pyspark.html#pyspark.RDD.top), [takeOrdered()](http://spark.apache.org/docs/latest/api/python/pyspark.html#pyspark.RDD.takeOrdered), and [reduce()](http://spark.apache.org/docs/latest/api/python/pyspark.html#pyspark.RDD.reduce)
#### Una cosa útil que hacer cuando tenemos un nuevo conjunto de datos es mirar las primeras entradas para obtener una idea aproximada de qué información está disponible. En Spark, podemos hacerlo usando las acciones `first ()`, `take ()`, `top ()` y `takeOrdered()`. Tenga en cuenta que para las acciones `first()` y `take()`, los elementos que se devuelven dependen de cómo el RDD es * particionado *.
#### En lugar de usar la acción `collect()`, podemos usar la acción `take(n)` para devolver los primeros n elementos del RDD. La acción `first()` devuelve el primer elemento de un RDD, y es equivalente a `take(1)`.
#### La acción `takeOrdered()` devuelve los primeros n elementos del RDD, usando su orden natural o un comparador personalizado. La principal ventaja de usar `takeOrdered()` en lugar de `first()` o `take()` es que `takeOrdered()` devuelve un resultado determinista, mientras que las otras dos acciones pueden devolver resultados diferentes dependiendo del número de particiones. `TakeOrdered()` devuelve la lista ordenada en * orden ascendente *. La acción `top()` es similar a `takeOrdered()`, excepto que devuelve la lista en * orden descendente. *
#### La acción `reduce()` reduce los elementos de un RDD a un solo valor aplicando una función que toma dos parámetros y devuelve un solo valor. La función debe ser conmutativa y asociativa, ya que `reduce()` se aplica en el nivel de partición y luego de nuevo para agregar los resultados de las particiones.

In [20]:
# Obtener el primer elemento
print filteredRDD.first()
# Obtener el cuatro elemento
print filteredRDD.take(4)
# se pueden pedir más elementos de los que tiene la coleccion,en este caso devuelve todos
print filteredRDD.take(12)

1
[1, 2, 3, 4]
[1, 2, 3, 4, 5, 6, 7, 8, 9]


In [21]:
# Obtiene los 3 elementos más pequeños
print filteredRDD.takeOrdered(3)
# Obtiene los 5 mayores elementos
print filteredRDD.top(5)

[1, 2, 3]
[9, 8, 7, 6, 5]


In [22]:
# Para obtener el orden  inverso podemos aplicar la función lambda
filteredRDD.takeOrdered(10, lambda s: -s)

[9, 8, 7, 6, 5, 4, 3, 2, 1]

In [23]:
# Operador suma
from operator import add
# Suma de valroes de un RDD
print filteredRDD.reduce(add)
# Suma de valores mediante función lambda
print filteredRDD.reduce(lambda a, b: a + b)

45
45


#### Aquí hay dos acciones adicionales que son útiles para recuperar información de un RDD: [takeSample ()] (http://spark.apache.org/docs/latest/api/python/pyspark.html#pyspark.RDD .takeSample) y [countByValue ()] (http://spark.apache.org/docs/latest/api/python/pyspark.html#pyspark.RDD.countByValue)
#### La acción `takeSample ()` devuelve una matriz con una muestra aleatoria de elementos del conjunto de datos. Requiere un argumento `withReplacement`, que especifica si está bien para elegir al azar el mismo elemento varias veces desde el RDD padre (por lo que cuando` withReplacement = True`, puede recuperar el mismo elemento varias veces). También toma un parámetro `seed` opcional que le permite especificar un valor de semilla para el generador de números aleatorios, de manera que se puedan obtener resultados reproducibles.
#### La acción `countByValue ()` devuelve el recuento de cada valor único en el RDD como un diccionario de tipo clave:valor

In [24]:
# obtiene 6 valores aleatorios reusando elementos
print filteredRDD.takeSample(withReplacement=True, num=6)
# obtiene 6 valores aleatorios sin reusar elementos
print filteredRDD.takeSample(withReplacement=False, num=6)

[4, 9, 3, 1, 2, 5]
[3, 9, 7, 6, 4, 5]


In [25]:
# Obtiene un diccionario del tipo clave:valor agrupando los valores
repetitiveRDD = sc.parallelize([1, 2, 3, 1, 2, 3, 1, 2, 1, 2, 3, 3, 3, 4, 5, 4, 6])
print repetitiveRDD.countByValue()

defaultdict(<type 'int'>, {1: 4, 2: 4, 3: 5, 4: 2, 5: 1, 6: 1})


#### ** `flatMap` **
#### Al realizar una transformación `map ()` usando una función, a veces la función devolverá más (o menos) que un elemento. Nos gustaría que el RDD recién creado consistiera en los elementos generados por la función. Simplemente aplicar una transformación `map ()` daría lugar a un nuevo RDD compuesto de iteradores. La solución es usar una transformación [flatMap ()] (http://spark.apache.org/docs/latest/api/python/pyspark.html#pyspark.RDD.flatMap), `flatMap ()` es similar a `Map ()`, excepto que con `flatMap ()` cada elemento de entrada se puede asignar a cero o más elementos de salida.

In [26]:
simpleRDD = sc.parallelize([2, 3, 4])
print simpleRDD.map(lambda x: range(1, x)).collect()
print simpleRDD.flatMap(lambda x: range(1, x)).collect()

[[1], [1, 2], [1, 2, 3]]
[1, 1, 2, 1, 2, 3]


#### ** `groupByKey` and `reduceByKey` **
#### [groupByKey()](http://spark.apache.org/docs/latest/api/python/pyspark.html#pyspark.RDD.groupByKey) and [reduceByKey()](http://spark.apache.org/docs/latest/api/python/pyspark.html#pyspark.RDD.reduceByKey)
#### Ambas de estas transformaciones operan en pares RDDs. Un par RDD es un RDD donde cada elemento es un par de tuplas (clave, valor). Por ejemplo, `sc.parallelize([('a', 1), ('a', 2), ('b', 1)])` crearía un par RDD donde las claves son 'a', 'a ',' b 'y los valores son 1, 2, 1.
#### La transformación `reduceByKey()` reúne pares que tienen la misma clave y aplica una función a dos valores asociados a la vez. `ReduceByKey()` funciona aplicando la función primero dentro de cada partición en una base per-key y luego a través de las particiones.
#### Si bien las transformaciones `groupByKey()` y `reduceByKey() a menudo se pueden usar para resolver el mismo problema y producir la misma respuesta, la transformación` reduceByKey () funciona mucho mejor para grandes conjuntos de datos distribuidos.
#### Mira el siguiente diagrama para entender cómo funciona `reduceByKey`. Observe cómo se combinan los pares en la misma máquina con la misma clave (utilizando la función lamdba pasada a reduceByKey) antes de que se barajen los datos. Entonces se llama de nuevo la función lamdba para reducir todos los valores de cada partición para producir un resultado final.
![reduceByKey() figure](http://spark-mooc.github.io/web-assets/images/reduce_by.png)
#### Por otra parte, cuando se utiliza la transformación `groupByKey()` - todos los pares clave-valor se barajan alrededor, causando una gran cantidad de datos innecesarios a ser transferidos a través de la red.
![groupByKey() figure](http://spark-mooc.github.io/web-assets/images/group_by.png)
#### A medida que su conjunto de datos crece, la diferencia en la cantidad de datos que hay que barajar, entre las transformaciones `reduceByKey()` y `groupByKey()`, se vuelve cada vez más exagerada.
#### Aquí hay más transformaciones que prefieren sobre `groupByKey()`:
   + #### [combineByKey ()] (http://spark.apache.org/docs/latest/api/python/pyspark.html#pyspark.RDD.combineByKey) se puede usar cuando se combinan elementos pero su retorno Tipo difiere de su tipo de valor de entrada.
   + #### [foldByKey ()] (http://spark.apache.org/docs/latest/api/python/pyspark.html#pyspark.RDD.foldByKey) fusiona los valores de cada tecla utilizando una función asociativa y Un "valor cero" neutro.
#### Ahora vayamos a través de un simple `groupByKey()` y `reduceByKey()` ejemplo.

In [27]:
pairRDD = sc.parallelize([('a', 1), ('a', 2), ('b', 1)])

# mapValues only used to improve format for printing
print pairRDD.groupByKey().mapValues(lambda x: list(x)).collect()

# reduceByKey es mas eficiente y escalable
print pairRDD.reduceByKey(add).collect()

[('b', [1]), ('a', [1, 2])]
[('b', 1), ('a', 3)]


### ** Cache y opciones de almacenamiento **

#### Por eficiencia Spark mantiene sus RDDs en memoria. Al mantener el contenido en la memoria, Spark puede acceder rápidamente a los datos. Sin embargo, la memoria es limitada, por lo que si intenta mantener demasiados RDD en la memoria, Spark eliminará automáticamente los RDD de la memoria para crear espacio para los nuevos RDD. Si más adelante se refiere a uno de los RDDs, Spark recreará automáticamente el RDD para usted, pero eso lleva tiempo.
#### Por lo tanto, si piensa usar un RDD más de una vez, entonces debe decirle a Spark que almacene en caché ese RDD. Puede utilizar la operación `cache ()` para mantener el RDD en la memoria. Sin embargo, si almacena en caché demasiados RDDs y Spark se queda sin memoria, eliminará primero el RDD que se haya utilizado menos recientemente(LRU). Una vez más, el RDD se recreará automáticamente cuando se acceda.
#### Puede comprobar si un RDD se almacena en caché utilizando el atributo `is_cached`

In [28]:
# Name the RDD
filteredRDD.setName('My Filtered RDD')
# Cache the RDD
filteredRDD.cache()
# Is it cached
print filteredRDD.is_cached

True
